##### ARTI 560 - Computer Vision  
## Image Classification using Transfer Learning - Exercise

### Objective

In this exercise, you will:

1. Select another pretrained model (e.g., VGG16, MobileNetV2, or EfficientNet) and fine-tune it for CIFAR-10 classification.  
You'll find the pretrained models in [Tensorflow Keras Applications Module](https://www.tensorflow.org/api_docs/python/tf/keras/applications).

2. Before training, inspect the architecture using model.summary() and observe:
- Network depth
- Number of parameters
- Trainable vs Frozen layers

3. Then compare its performance with ResNet and the custom CNN.

### Questions:

- Which model achieved the highest accuracy?
- Which model trained faster?
- How might the architecture explain the differences?

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

In [2]:

# -----------------------------
# 1) Load CIFAR-10
# -----------------------------
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

class_names = [
    "airplane","automobile","bird","cat","deer",
    "dog","frog","horse","ship","truck"
]

# Keep labels as integers (SparseCategoricalCrossentropy)
y_train = y_train.squeeze().astype("int64")
y_test  = y_test.squeeze().astype("int64")

# Convert images to float32
x_train = x_train.astype("float32")
x_test  = x_test.astype("float32")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


In [3]:
# -----------------------------
# 2) Data augmentation
# -----------------------------
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
], name="augmentation")


In [4]:
# -----------------------------
# 3) Build EfficientNetB0 backbone (pretrained) without classifier layer.
# -----------------------------
efficientnet_base = keras.applications.EfficientNetB0(
    include_top=False,  # Exclude classifier layer for transfer learning
    weights="imagenet",
    input_shape=(96, 96, 3)  # Reduce input size to save memory
)

efficientnet_base.trainable = True  # Unfreeze for fine-tuning

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [5]:
# -----------------------------
# 4) Full model (preprocess inside model)
# -----------------------------
efficientnet_model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    data_augmentation,
    layers.Resizing(96, 96, interpolation="bilinear"),  # Match EfficientNetB0 input
    layers.Lambda(preprocess_input),
    efficientnet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10)  # logits
], name="cifar10_efficientnetb0")

efficientnet_model.summary()

Model: "cifar10_efficientnetb0"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augmentation (Sequential)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing (Resizing)             │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 3, 3, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,062,381 (15.50 MB)

 Trainable params: 4,020,358 (15.34 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [6]:

# -----------------------------
# 5) Compile + Train (frozen backbone)
# -----------------------------
efficientnet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=1),
]





In [8]:
history = efficientnet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=10,
    batch_size=34,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/10
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 162s 109ms/step - accuracy: 0.7598 - loss: 0.7023 - val_accuracy: 0.8650 - val_loss: 0.4011 - learning_rate: 0.0010
Epoch 2/10
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 141s 107ms/step - accuracy: 0.8610 - loss: 0.4061 - val_accuracy: 0.8854 - val_loss: 0.3474 - learning_rate: 0.0010
Epoch 3/10
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 141s 107ms/step - accuracy: 0.8796 - loss: 0.3480 - val_accuracy: 0.8812 - val_loss: 0.3638 - learning_rate: 0.0010
Epoch 4/10
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 141s 107ms/step - accuracy: 0.9166 - loss: 0.2356 - val_accuracy: 0.9196 - val_loss: 0.2460 - learning_rate: 5.0000e-04
Epoch 5/10
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 141s 106ms/step - accuracy: 0.9346 - loss: 0.1878 - val_accuracy: 0.9198 - val_loss: 0.2363 - learning_rate: 5.0000e-04
Epoch 6/10
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 141s 107ms/step - accuracy: 0.9446 - loss: 0.1628 - val_accuracy: 0.9278 - val_loss: 0.2315 - learning_rate: 5.0000e-04
Epoch 7/10
1324/1324 ━━━━━━━━━━━━━━━━━━━

In [9]:
test_loss_ft, test_acc_ft = efficientnet_model.evaluate(x_test, y_test, verbose=0)
print("EfficientNetB0 (fine-tuned) test accuracy:", test_acc_ft)
print("EfficientNetB0 (fine-tuned) test loss    :", test_loss_ft)

EfficientNetB0 (fine-tuned) test accuracy: 0.9361000061035156
EfficientNetB0 (fine-tuned) test loss    : 0.2147226631641388
